In [1]:
import numpy as np

from psikit import Psikit
from psiomm import molecule
from psiomm import psi_omm as po

from simtk.openmm.app import Simulation
from simtk.openmm.app import ForceField
from simtk.openmm.app import Modeller
from simtk.openmm.app import PDBReporter

from simtk.openmm import Platform
from simtk.openmm import LangevinIntegrator

from simtk.openmm import Vec3

from simtk.unit import picosecond
from simtk.unit import kelvin
from simtk.unit import kilocalorie_per_mole
from simtk.unit import angstrom

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole

RDKit WARNING: [09:11:58] Enabling RDKit 2019.09.3 jupyter extensions


## Work flow 

- make molecule from xyz string => solute
- generate bonds
- generate atom types
- generate charges

- make topology
- define forcefield
- generate template with FF, topology and solute
- make modeller
- perform simulation to get trajectory

In [2]:
pk = Psikit()

In [3]:
pk.read_from_smiles('CCOc1cccnc1')

In [4]:
pk.rdkit_optimize()

In [5]:
pk.mol.GetNumConformers()

1

In [6]:
# remove mutiplicity, formal charge
xyzstring = '\n'.join(pk.mol2xyz().split('\n')[2:])

In [7]:
solute = molecule.Molecule.from_xyz_string(xyzstring)

In [8]:
solute.generate_bonds()
solute.generate_atom_types()
solute.generate_charges()

In [9]:
# check generated atoms and charges
for i, at in enumerate(solute.atom_types):
    print(i, at, solute.charges[i])

0 c3 -0.16240415191041357
1 c3 0.03679384783580364
2 os -0.263056064493707
3 ca 0.130662401965024
4 ca -0.0752094501302798
5 ca -0.05835160068850964
6 ca 0.017895212168927088
7 nb -0.2358970396079183
8 ca -0.0012324033472115303
9 hc 0.06626046228444116
10 hc 0.06408533045243181
11 hc 0.0662604628830285
12 h1 0.05665766362774605
13 h1 0.056657585826414836
14 ha 0.07811395098040658
15 ha 0.07313080623699353
16 h4 0.0746315390071699
17 h4 0.07500144690962385


In [10]:
topology = po.make_topology(solute)

In [11]:
forcefield = ForceField('gaff2.xml', 'tip3p.xml')

In [12]:
po.generateTemplate(forcefield, topology, solute)

In [13]:
coords = []
for i in range(len(solute.xyz)):
    print(solute.xyz[i])
    coords.append(Vec3(solute.xyz[i][0], solute.xyz[i][1], solute.xyz[i][2]))
positions = coords * angstrom

[ 3.24816249 -0.28545284 -0.04993103]
[1.92071005 0.42919173 0.14531796]
[ 0.85091939 -0.4907528  -0.05535284]
[-0.50768098 -0.15878488  0.04887394]
[-1.46213108 -1.15677838 -0.17258026]
[-2.8230515  -0.8610512  -0.07698647]
[-3.21883223  0.43887336  0.24135335]
[-2.28344822  1.40174908  0.45475069]
[-0.95180364  1.13851888  0.36746672]
[ 3.34237371 -1.11698822  0.68025764]
[4.08472027 0.42799293 0.10557298]
[ 3.31027413 -0.69564551 -1.08016704]
[1.87654003 0.83881484 1.17771217]
[ 1.84434812  1.26134088 -0.58767369]
[-1.14759338 -2.16296406 -0.41913518]
[-3.56354787 -1.63168544 -0.24792887]
[-4.26989979  0.68286051  0.31891397]
[-0.2500595   1.94076113  0.54667875]


In [14]:
angstrom

Unit({BaseUnit(base_dim=BaseDimension("length"), name="angstrom", symbol="A"): 1.0})

In [15]:
coords

[Vec3(x=3.2481624923230448, y=-0.2854528425796883, z=-0.04993103005605603),
 Vec3(x=1.920710052723795, y=0.4291917313962072, z=0.1453179551578719),
 Vec3(x=0.8509193877262783, y=-0.49075280476915223, z=-0.055352838733997316),
 Vec3(x=-0.5076809807429097, y=-0.1587848841626658, z=0.04887394206217463),
 Vec3(x=-1.4621310768713758, y=-1.1567783763036417, z=-0.1725802617719825),
 Vec3(x=-2.8230514983120987, y=-0.8610511955441892, z=-0.0769864701930148),
 Vec3(x=-3.2188322259953295, y=0.438873361620278, z=0.24135335156400248),
 Vec3(x=-2.283448224222744, y=1.4017490836032298, z=0.4547506919670112),
 Vec3(x=-0.9518036417405741, y=1.1385188763514051, z=0.36746672198889957),
 Vec3(x=3.3423737135879765, y=-1.1169882182841946, z=0.6802576399828909),
 Vec3(x=4.084720272244058, y=0.4279929337767665, z=0.10557297623138764),
 Vec3(x=3.3102741303712704, y=-0.6956455148747527, z=-1.0801670357578719),
 Vec3(x=1.8765400251897808, y=0.8388148365810248, z=1.1777121658397807),
 Vec3(x=1.8443481168007743, y

In [16]:
positions

Quantity(value=[Vec3(x=3.2481624923230448, y=-0.2854528425796883, z=-0.04993103005605603), Vec3(x=1.920710052723795, y=0.4291917313962072, z=0.1453179551578719), Vec3(x=0.8509193877262783, y=-0.49075280476915223, z=-0.055352838733997316), Vec3(x=-0.5076809807429097, y=-0.1587848841626658, z=0.04887394206217463), Vec3(x=-1.4621310768713758, y=-1.1567783763036417, z=-0.1725802617719825), Vec3(x=-2.8230514983120987, y=-0.8610511955441892, z=-0.0769864701930148), Vec3(x=-3.2188322259953295, y=0.438873361620278, z=0.24135335156400248), Vec3(x=-2.283448224222744, y=1.4017490836032298, z=0.4547506919670112), Vec3(x=-0.9518036417405741, y=1.1385188763514051, z=0.36746672198889957), Vec3(x=3.3423737135879765, y=-1.1169882182841946, z=0.6802576399828909), Vec3(x=4.084720272244058, y=0.4279929337767665, z=0.10557297623138764), Vec3(x=3.3102741303712704, y=-0.6956455148747527, z=-1.0801670357578719), Vec3(x=1.8765400251897808, y=0.8388148365810248, z=1.1777121658397807), Vec3(x=1.8443481168007743,

In [17]:
modeller = Modeller(topology, positions)
modeller.addSolvent(forcefield, numAdded=50)

In [18]:
omm_sys = forcefield.createSystem(modeller.topology)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picosecond)

In [19]:
simulation = Simulation(modeller.topology,
                       omm_sys,
                       integrator,
                       platform=Platform.getPlatformByName('Reference'))

In [20]:
simulation.context.setPositions(modeller.positions)
simulation.reporters.append(PDBReporter('output.pdb', 10))

In [21]:
simulation.step(400)

In [22]:
simulation.minimizeEnergy(tolerance=2*kilocalorie_per_mole)

In [30]:
new_z_vals, new_xyz = po.get_atom_positions(simulation.topology, simulation)
po.write_traj('mol-water_opt_geo_mm.xyz', new_z_vals, new_xyz, comment="Optimized Geometry")

In [25]:
new_mol = molecule.Molecule(new_z_vals, new_xyz)

In [26]:
pk.psi4.core.clean()

In [34]:
# The string has molecule and 50 water molecules.
# it seems not good input format for psi4 (IMHO)
pk.psi4.geometry(new_mol.to_xyz_string())

In [35]:
print(new_mol.to_xyz_string())

  C    2.107668    -1.041729     0.407400
  C    0.939056    -0.080194     0.152568
  O   -0.262383    -0.862826     0.045346
  C   -1.487680    -0.296092    -0.247091
  C   -2.621257    -1.120669    -0.295859
  C   -3.859757    -0.541903    -0.609772
  C   -3.918938     0.838723    -0.863944
  N   -2.834127     1.639124    -0.823689
  C   -1.642206     1.081641    -0.522831
  H    1.941580    -1.627038     1.319255
  H    3.045004    -0.484339     0.522443
  H    2.228201    -1.736929    -0.429190
  H    0.856149     0.637291     0.978373
  H    1.119733     0.470401    -0.777223
  H   -2.540967    -2.184617    -0.096930
  H   -4.758064    -1.149656    -0.659803
  H   -4.864366     1.317780    -1.113085
  H   -0.796466     1.766584    -0.499850
  O   -1.031373     2.089310    -3.782707
  H   -1.582826     2.865109    -3.681382
  H   -0.257056     2.274190    -3.251215
  O    0.926622    -0.314881     3.574484
  H    0.800566    -0.107839     4.500483
  H    1.875499    -0.388005     3